# pass@k analysis
This notebook demonstrates how to visualize which model managed to achieve which pass@k value.

In [ ]:
import pandas as pd
import os

In [ ]:
directory = "../data/"

collection = []
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        print(filename)
        df = pd.read_csv(directory + filename)
        df['model'] = filename.replace("_passk_results.csv", "").replace("samples_", "")
        collection.append(df)

df = pd.concat(collection)

In [ ]:
df